<a href="https://colab.research.google.com/github/Stephanie-DS/takehome-stephanie-workingcopy/blob/main/embeddings_regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install pytorch_pretrained_bert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 747.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 2.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-man

In [ ]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

# loading model tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
#%% import data
#import re

url = 'https://raw.githubusercontent.com/Pivotal-Life-Sciences-DS/takehome-stephanie/main/askscience_data.csv'
df = pd.read_csv(url)
df = df[df.score!=df.score.max()]
df = df[~df.tag.isin(['Meta', 'META'])]

df = df[['title', 'body', 'score']]

#%% I'm doing an easy version and breaking the scores into quantiles.
df['quants'] = pd.qcut(df['score'], 5, labels=False)

# add a text field with the BERT separators
text = '[CLS]' + df['title'] + '[SEP]' + df['body'].fillna(" ") + '[SEP]'
#print(text.head())

marked_text = []
# Now tokenize it using BERT tokenizers
for i in text:
  tokenized_text = tokenizer.tokenize(i)
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  marked_text.append(tokenized_text)

df['text'] = marked_text

df.drop(['score', 'title', 'body'], axis=1, inplace=True)